In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import gzip
import io
from PIL import Image
import random

In [2]:
def process_bed(directory, filename):
        
        print(directory)
        print(filename)
        
        f = os.path.join(directory, filename)
        with gzip.open(f, 'rb') as file:
            file_content = file.read().decode('utf-8')  # Decode the bytes to string

        # Process the file content and convert it into a DataFrame
        # Assuming the file content is in a specific format, modify the code accordingly
        # For example, if the content is a CSV file, you can use pd.read_csv()
        # If it's in JSON format, you can use pd.read_json(), etc.
        df = pd.read_csv(io.BytesIO(file_content.encode()), delimiter='\t')  # Use BytesIO to read from bytes
      # Use StringIO to read from a string
        chrom_pos={'chr1': 0,
            'chr2': 248956422,
            'chr3': 491149951,
            'chr4': 689445510,
            'chr5': 879660065,
            'chr6': 1061198324,
            'chr7': 1232004303,
            'chr8': 1391350276,
            'chr9': 1536488912,
            'chr10': 1674883629,
            'chr11': 1808681051,
            'chr12': 1943767673,
            'chr13': 2077042982,
            'chr14': 2191407310,
            'chr15': 2298451028,
            'chr16': 2400442217,
            'chr17': 2490780562,
            'chr18': 2574038003,
            'chr19': 2654411288,
            'chr20': 2713028904,
            'chr21': 2777473071,
            'chr22': 2824183054,
            'chrX': 2875001522,
            'chrY': 3031042417}

        header = ['chrom', 'read_start', 'read_end', 'name', 'score', 'strand']
        df.columns = header[:len(df.columns)]

        max_value = df['read_end'].max()
        df['name'] = df['name'].str.rstrip('12') + '1'
        df['frag_length']=df.groupby('name')['read_end'].transform('max') - df.groupby('name')['read_start'].transform('min')
        df = df.groupby('name',as_index=False).agg({
        'chrom': 'first',
        'read_start': 'min',
        'read_end': 'max',
        'name':'first',
        'score': 'first',
        'strand': 'first',
        'frag_length':'first'
        })
   

        df["chrom"] = df["chrom"].str.split("_").str.get(0)
        mask = df["chrom"].str.startswith("chrUn")
    
        # Filter the DataFrame to exclude rows with "chrUn" chromosomes
        df= df[~mask]
        df = df[df.frag_length <= 1000]
        df['frag_length'].max()

        #make sure the positions are correct 
        for key, value in chrom_pos.items():
    
            mask = df['chrom'] == key
        
            df.loc[mask, 'read_start'] += value
            df.loc[mask,'read_end']+=value
        
        #convert columns
        columns_to_convert = ['read_start', 'read_end','frag_length']
        df[columns_to_convert] = df[columns_to_convert].astype(int)
        df=df.sort_values('read_start')
        df=df.drop_duplicates(subset=['read_start', 'read_end'], keep='first')


        num_bins = 2000000
        bin_edges = np.linspace(0, 3088269832, num_bins + 1)
        hist, bins = np.histogram(np.concatenate([df['read_start'].values, df['read_end'].values]), bins=bin_edges)
 
        return hist

In [3]:
trying=process_bed('/home/sam/sra_files','cancer_SRR17006162.bed.gz')

/home/sam/sra_files
cancer_SRR17006162.bed.gz


In [4]:
cancer,control=[],[]
# Set the number of patients and batch size
directory='/home/sam/sra_files'
files=os.listdir(directory)
print(files[0:10])
for k in files:
    if 'cancer' in k and len(cancer)<400:
        cancer.append(k)
    elif 'control' in k and len(control)<400:
        control.append(k)
print(len(cancer))

new_files=cancer+control
print(cancer[0:10])
print(len(new_files))

['control_SRR16993778.bed.gz', 'control_SRR16994988.bed.gz', 'cancer_SRR16993515.bed.gz', 'control_SRR16994077.bed.gz', 'control_SRR16994505.bed.gz', 'control_SRR16993781.bed.gz', 'cancer_SRR16995157.bed.gz', 'cancer_SRR16993788.bed.gz', 'cancer_SRR16993797.bed.gz', 'control_SRR16993531.bed.gz']
424
['cancer_SRR16993515.bed.gz', 'cancer_SRR16995157.bed.gz', 'cancer_SRR16993788.bed.gz', 'cancer_SRR16993797.bed.gz', 'cancer_SRR17004200.bed.gz', 'cancer_SRR16994067.bed.gz', 'cancer_SRR16994348.bed.gz', 'cancer_SRR16994206.bed.gz', 'cancer_SRR16994406.bed.gz', 'cancer_SRR16993931.bed.gz']
824


In [15]:
with open(r'/home/sam/cancer_names.txt', 'w') as fp:
    for item in cancer:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

with open(r'/home/sam/control_names.txt', 'w') as fp:
        for item in control:
            # write each item on a new line
            fp.write("%s\n" % item)
        print('Done')

(400, 2000000)


In [5]:

cancer_full,control_full=[],[]
l=0
for k in new_files:
    if 'cancer' in k:
        patient=process_bed(directory,k)
        cancer_full.append(patient)
    elif 'control' in k:
        patient=process_bed(directory,k)
        control_full.append(patient)
        
    if len(control_full)==400:
        control_arr=np.array(control_full)
        with open('test_sort.npy', 'ab') as f:
            np.save(f, control_arr)
        control_arr=[]
        control_full=[]
    elif len(cancer_full)==400:
        cancer_arr=np.array(cancer_full)
        with open('test_sort.npy', 'wb') as f:
            np.save(f, cancer_arr)
        cancer_arr=[]
        cancer_full=[]
    l+=1
    if l%10==0:
        print(l)


/home/sam/sra_files
cancer_SRR16993515.bed.gz
/home/sam/sra_files
cancer_SRR16995157.bed.gz
/home/sam/sra_files
cancer_SRR16993788.bed.gz
/home/sam/sra_files
cancer_SRR16993797.bed.gz
/home/sam/sra_files
cancer_SRR17004200.bed.gz
/home/sam/sra_files
cancer_SRR16994067.bed.gz
/home/sam/sra_files
cancer_SRR16994348.bed.gz
/home/sam/sra_files
cancer_SRR16994206.bed.gz
/home/sam/sra_files
cancer_SRR16994406.bed.gz
/home/sam/sra_files
cancer_SRR16993931.bed.gz
10
/home/sam/sra_files
cancer_SRR16994852.bed.gz
/home/sam/sra_files
cancer_SRR16994022.bed.gz
/home/sam/sra_files
cancer_SRR17004844.bed.gz
/home/sam/sra_files
cancer_SRR17004491.bed.gz
/home/sam/sra_files
cancer_SRR16994379.bed.gz
/home/sam/sra_files
cancer_SRR16994978.bed.gz
/home/sam/sra_files
cancer_SRR16994762.bed.gz
/home/sam/sra_files
cancer_SRR16993862.bed.gz
/home/sam/sra_files
cancer_SRR17004630.bed.gz
/home/sam/sra_files
cancer_SRR16993591.bed.gz
20
/home/sam/sra_files
cancer_SRR16994932.bed.gz
/home/sam/sra_files
cancer_S